In [3]:
%matplotlib inline
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns 
import plotly.express as px
from plotly.offline import iplot, plot
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report 
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import cross_val_score

read the data

In [4]:
df.to_csv('data.csv',index=False)
df = pd.read_csv('data.csv')
df.info()

NameError: name 'df' is not defined

总共30个特征，分别是对10个实值特征计算，mean, se, worst
diagnosis 列为标签

In [ ]:
plt.rcParams['figure.figsize'] = (15,8) 
plt.rcParams['axes.titlesize'] = 'large'
df = pd.read_csv('data.csv')
#basic descriptive statistics
df.iloc[:,2:32].describe()

3-32 列包含 30 个实值特征，可用于构建模型来预测肿瘤是良性还是恶性。

In [ ]:
diagnosis = df['diagnosis'].value_counts().reset_index()
diagnosis

In [ ]:
df.isnull().sum()

无缺失值

In [ ]:
df.diagnosis.value_counts().plot(kind = "bar")

良性：恶性约为2：1

数据探索性分析
数据探索主要是掌握，数据的结构，值的分布，在数据集中是否存在异常值，特征间相互关系。
主要包括描述性统计分析和
数据可视化

描述性统计

In [ ]:
#Load libraries for data processing
import pandas as pd 
from scipy.stats import norm
import seaborn as sns # visualization

plt.rcParams['figure.figsize'] = (15,8) 
plt.rcParams['axes.titlesize'] = 'large'

data = pd.read_csv('data.csv')
#basic descriptive statistics
data.iloc[:,2:32].describe()

In [ ]:
df.columns

In [ ]:
Dimensions = df[['radius_mean', 'texture_mean', 'perimeter_mean',
       'area_mean', 'smoothness_mean', 'compactness_mean', 'concavity_mean',
       'concave points_mean', 'symmetry_mean', 'fractal_dimension_mean']].sum().reset_index()
Dimensions

数据可视化

In [ ]:
sns.set_style("white")
sns.set_context({"figure.figsize": (8, 8)})

将特征分为三组：mean, se, worst化

In [ ]:
#For a merge + slice:
data_mean=df.iloc[:,2:12]
data_se=df.iloc[:,12:22]
data_worst=df.iloc[:,22:32]
print(data_mean.columns)
print(data_se.columns)
print(data_worst.columns)

各组特征可视化 -- 直方图

In [ ]:
#Plot histograms of CUT1 variables
data_mean.hist(bins=10, figsize=(15, 10),grid=False,color = "pink")
data_se.hist(bins=10, figsize=(15, 10),grid=False,color = "orange")
data_worst.hist(bins=10, figsize=(15, 10),grid=False,color = "blue")

可以看到，属性 凹度， 凹点 可能具有指数分布；纹理，平滑，对称属性可能具有高斯或接近高斯分布。

概率密度曲线

In [ ]:
plt = data_mean.plot(kind= 'density', subplots=True, 
                     layout=(4,3), sharex=False, 
                     sharey=False, fontsize=15, figsize=(15,10))

plt = data_se.plot(kind= 'density', subplots=True, 
                     layout=(4,3), sharex=False, 
                     sharey=False, fontsize=15, figsize=(15,10))

plt = data_worst.plot(kind= 'density', subplots=True, 
                     layout=(4,3), sharex=False, 
                     sharey=False, fontsize=15, figsize=(15,10))

周长、半径、面积、凹度、密度可能具有指数分布； 纹理、平滑、对称属性可能具有高斯或接近高斯分布。

指数分布 的特征经对数log()变换之后可以呈高斯分布

In [ ]:
# transform exponential distribution to Gaussian univariate distribution
data_mean['area_mean'].plot(kind = "hist", figsize=(8,6))

In [ ]:
np.log1p(data_mean['area_mean']).plot(kind = "hist", figsize=(8,6))

In [ ]:
np.log10(data_mean['area_mean']).plot(kind = "hist", figsize=(8,6))

通过箱线图可视化数据分布情况和异常值

In [ ]:
# box and whisker plots
plt=data_mean.plot(kind= 'box' , subplots=True, layout=(4,4), 
                   sharex=False, sharey=False,fontsize=12)

plt=data_se.plot(kind= 'box' , subplots=True, layout=(4,4), 
                 sharex=False, sharey=False,fontsize=12)

plt=data_worst.plot(kind= 'box' , subplots=True, layout=(4,4), 
                 sharex=False, sharey=False,fontsize=12)

多模态数据可视化：散点图和相关矩阵

In [ ]:
# plot correlation matrix
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt

plt.style.use('fivethirtyeight')
sns.set_style("white")
# Compute the correlation matrix
corr = data_mean.corr()
# Generate a mask for the upper triangle
mask = np.zeros_like(corr, dtype=np.bool_)
mask[np.triu_indices_from(mask)] = True
# Set up the matplotlib figure
data, ax = plt.subplots(figsize=(8, 8))
plt.title('Breast Cancer Feature Correlation---liziteng')
# Generate a custom diverging colormap
cmap = sns.diverging_palette(260, 10, as_cmap=True)
# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, vmax=1.2, square='square', cmap=cmap, mask=mask, 
            ax=ax,annot=True, fmt='.2g',linewidths=2)

可以看到平均值参数在 1-0.75 之间存在很强的正相关关系；。
组织核的平均面积与半径和参数的均值呈强正相关；
一些参数中度正相关（r在0.5-0.75之间）是凹度和面积，凹度和周长等; 同样，我们看到 fractal_dimension 与半径、纹理、参数平均值之间存在一些强烈的负相关。

In [ ]:
data = pd.read_csv("data.csv")
g = sns.PairGrid(data[data.columns.tolist()[1:12]],
                 hue ='diagnosis')
g = g.map_diag(plt.hist)
g = g.map_offdiag(plt.scatter, s = 3)

预处理与特征工程

可以看到，大多数特征对于肿瘤良恶性的区分度还是很大的。

小结：
细胞半径、周长、面积、紧密度、凹度和凹点的平均值可用于癌症的分类。 这些参数的较大值倾向于显示与恶性肿瘤的相关性。
质地、平滑度、对称性或分维数的平均值并未显示出较好的诊断偏好。
在任何直方图中，都没有明显的异常值需要进一步清理。

预处理和特征工程

数据加载

In [ ]:
plt.style.use('fivethirtyeight')
sns.set_style("white")
plt.rcParams['figure.figsize'] = (8,4) 
data = pd.read_csv('data.csv', index_col=False)

划分训练集和测试集

In [ ]:
X = data.iloc[:,2:32]
y = data.iloc[:,1].apply(lambda x: 1 if x == "M" else 0)
##train 70% test 30%
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=7)
X_train.shape, y_train.shape, X_test.shape, y_test.shape

数据标准化

In [ ]:
# 归一化数据，以0为中心，缩放以消除方差
scaler =StandardScaler()
Xs = scaler.fit_transform(X)

PCA降维

In [ ]:
# 从 30维 降到 10维
pca = PCA(n_components=10)
fit = pca.fit(Xs)
X_pca = pca.transform(Xs)

取前两个PC 进行画图，查看降维后特征的区分度

In [ ]:
PCA_df = pd.DataFrame()
PCA_df['PCA_1'] = X_pca[:,0]
PCA_df['PCA_2'] = X_pca[:,1]
## 可视化
plt.figure(figsize=(8,6))
plt.plot(PCA_df['PCA_1'][data.diagnosis == 'M'],
         PCA_df['PCA_2'][data.diagnosis == 'M'],
         'o', alpha = 0.7, color = 'r')
plt.plot(PCA_df['PCA_1'][data.diagnosis == 'B'],
         PCA_df['PCA_2'][data.diagnosis == 'B'],
         'o', alpha = 0.7, color = 'b')

plt.xlabel('PCA_1')
plt.ylabel('PCA_2')
plt.legend(['Malignant','Benign'])
plt.show()

通过拐点，确定选择前几个主成分用于后续建模

In [ ]:
#The amount of variance that each PC explains
var = pca.explained_variance_ratio_
### 通过拐点确定选择前几个PC
plt.plot(var)
plt.title('Scree Plot')
plt.xlabel('Principal Component')
plt.ylabel('Eigenvalue')

leg = plt.legend(['Eigenvalues from PCA'], 
                 loc='best', 
                 borderpad=0.3,
                 shadow=False,
                 markerscale=0.4)

leg.get_frame().set_alpha(0.4)


plt.show()

In [ ]:
DecisionTree = DecisionTreeClassifier()

In [ ]:
DecisionTree.fit(X_train,y_train)

In [ ]:
DecisionTree.score(X_train,y_train)

通过测试集对模型进行评分

In [ ]:
DecisionTree.score(X_test,y_test)

In [ ]:
y_pred = DecisionTree.predict(X_test)
y_pred

In [ ]:
sns.heatmap(confusion_matrix(y_test,y_pred), annot=True, cmap='viridis', fmt=".2f")

In [ ]:
print(classification_report(y_test,y_pred))

交叉验证

交叉验证5次，每次随机取4/5的数据用于训练，1/5的数据用于测试。如果不设置该参数，则默认交叉验证3次
以默认值’accuracy’（准确度）作为评估标准

In [ ]:
acc=cross_val_score(DecisionTree,X,y,cv=5)
acc

In [ ]:
print(acc.mean())

In [ ]:
# 计算不同阈值下的真正例率（TPR）和假正例率（FPR）
from sklearn.metrics import roc_curve, auc
# 预测测试集的概率
y_score = DecisionTree.predict_proba(X_test)[:, 1]
# 计算不同阈值下的真正例率（TPR）和假正例率（FPR）
fpr, tpr, thresholds = roc_curve(y_test, y_score)
roc_auc = auc(fpr, tpr)

# 绘制ROC曲线
plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characterisitc---liziteng')
plt.legend(loc="lower right")
plt.show()

AUC值

In [ ]:
from sklearn.metrics import roc_auc_score
roc_auc = auc(fpr, tpr)
print(roc_auc)

以ROC曲线的AUC值作为评估标准，设置scoring参数为’roc_auc’

In [ ]:
acc=cross_val_score(DecisionTree,X,y,scoring='roc_auc',cv=5)
acc

In [ ]:
print(acc.mean())

AUC值偏低，尝试一下参数调优

GridSearch网格搜索
GridSearch网格搜索是一种穷举搜索的参数调优手段：遍历所有的候选参数，循环建立模型并评估模型的有效性和准确性，选取表现最好的参数作为最终结果。

多参数调优

In [ ]:
from sklearn.model_selection import GridSearchCV
parameters={'max_depth':[1,2,3,4,5,6,7,8,9],'criterion':['gini','entropy'],'min_samples_split':[5,6,7,8,9,11,12,13,14,15]}
grid_search=GridSearchCV(DecisionTree,parameters,scoring='roc_auc',cv=5)
grid_search.fit(X_train,y_train)
grid_search.best_params_

In [ ]:
DecisionTree=DecisionTreeClassifier(criterion='entropy',max_depth=2,min_samples_split=5)
DecisionTree.fit(X_train,y_train)
DecisionTree.score(X_train,y_train)

In [ ]:
DecisionTree.score(X_test,y_test)

In [ ]:
# 计算不同阈值下的真正例率（TPR）和假正例率（FPR）
from sklearn.metrics import roc_curve, auc
# 预测测试集的概率
y_score = DecisionTree.predict_proba(X_test)[:, 1]
# 计算不同阈值下的真正例率（TPR）和假正例率（FPR）
fpr, tpr, thresholds = roc_curve(y_test, y_score)
roc_auc = auc(fpr, tpr)

# 绘制ROC曲线
plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([-0.1, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characterisitc---liziteng')
plt.legend(loc="lower right")
plt.show()

AUC值

In [ ]:
from sklearn.metrics import roc_auc_score
roc_auc = auc(fpr, tpr)
print(roc_auc)

单参数调优

参数max_depth

In [ ]:
parameters={'max_depth':[1,2,3,4,5,6,7,8,9]}
grid_search=GridSearchCV(DecisionTree,parameters,scoring='roc_auc',cv=5)
grid_search.fit(X_train,y_train)
grid_search.best_params_

In [ ]:
DecisionTree=DecisionTreeClassifier(max_depth=2)
DecisionTree.fit(X_train,y_train)
DecisionTree.score(X_train,y_train)

In [ ]:
DecisionTree.score(X_test,y_test)

In [ ]:
y_score = DecisionTree.predict_proba(X_test)[:, 1]
# 计算不同阈值下的真正例率（TPR）和假正例率（FPR）
fpr, tpr, thresholds = roc_curve(y_test, y_score)
roc_auc = auc(fpr, tpr)

# 绘制ROC曲线
plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characterisitc---liziteng')
plt.legend(loc="lower right")
plt.show()

AUC值

In [ ]:
from sklearn.metrics import roc_auc_score
roc_auc = auc(fpr, tpr)
print(roc_auc)

参数min_samples_spilt

In [ ]:
parameters={'min_samples_split':[11,12,13,14,15,16]}
grid_search=GridSearchCV(DecisionTree,parameters,scoring='roc_auc',cv=5)
grid_search.fit(X_train,y_train)
grid_search.best_params_

In [ ]:
DecisionTree=DecisionTreeClassifier(min_samples_split=16)
DecisionTree.fit(X_train,y_train)
DecisionTree.score(X_train,y_train)

In [ ]:
DecisionTree.score(X_test,y_test)

In [ ]:
y_score = DecisionTree.predict_proba(X_test)[:, 1]
# 计算不同阈值下的真正例率（TPR）和假正例率（FPR）
fpr, tpr, thresholds = roc_curve(y_test, y_score)
roc_auc = auc(fpr, tpr)

# 绘制ROC曲线
plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characterisitc---liziteng')
plt.legend(loc="lower right")
plt.show()

AUC值

In [ ]:
from sklearn.metrics import roc_auc_score
roc_auc = auc(fpr, tpr)
print(roc_auc)

参数criterion

In [ ]:
DecisionTree=DecisionTreeClassifier(criterion='entropy')
DecisionTree.fit(X_train,y_train)
DecisionTree.score(X_train,y_train)

In [ ]:
DecisionTree.score(X_test,y_test)

In [ ]:
y_score = DecisionTree.predict_proba(X_test)[:, 1]
# 计算不同阈值下的真正例率（TPR）和假正例率（FPR）
fpr, tpr, thresholds = roc_curve(y_test, y_score)
roc_auc = auc(fpr, tpr)

# 绘制ROC曲线
plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characterisitc---liziteng')
plt.legend(loc="lower right")
plt.show()

AUC值

In [ ]:
from sklearn.metrics import roc_auc_score
roc_auc = auc(fpr, tpr)
print(roc_auc)

以调整特征选择标准为信息熵后的ROC曲线的AUC值作为评估标准，设置scoring参数为’roc_auc’

In [ ]:
acc=cross_val_score(DecisionTree,X,y,scoring='roc_auc',cv=5)
acc

In [ ]:
print(acc.mean())

In [ ]:
y_pred = DecisionTree.predict(X_test)
y_pred

In [ ]:
sns.heatmap(confusion_matrix(y_test,y_pred), annot=True, cmap='viridis', fmt=".2f")

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import KFold 
import warnings

def bxplots(results,names):
    fig = plt.figure()
    fig.suptitle('Algorithm Comparison')
    ax = fig.add_subplot(111)
    plt.boxplot(results)
    ax.set_xticklabels(names)
    plt.show()

def piplinecompare(models, X_train, y_train):
    results = []
    names = []
    for name, model in models:
        kfold = KFold(shuffle=True,n_splits=10,random_state=7)
        cv_results = cross_val_score(model, X_train, y_train, cv=kfold, scoring='roc_auc')
        results.append(cv_results)
        names.append(name)
        msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
        print(msg)
    return results,names
pipelines = []
models = []
#lbfgs算法进行模型训练时，迭代过程没有收敛，会出现警告信息,忽略警告信息
warnings.filterwarnings("ignore", category=Warning)
models.append(('LR', LogisticRegression()))
models.append(('LDA',LinearDiscriminantAnalysis()))
models.append(('KNN',KNeighborsClassifier()))
models.append(('CART',DecisionTree))
models.append(('NB',GaussianNB()))
models.append(('SVM',SVC()))
pipelines.append(( 'ScaledLR' , Pipeline([( 'Scaler' , StandardScaler()),( 'LR' ,
    LogisticRegression())])))
pipelines.append(( 'ScaledLDA' , Pipeline([( 'Scaler' , StandardScaler()),( 'LDA' ,
    LinearDiscriminantAnalysis())])))
pipelines.append(( 'ScaledKNN' , Pipeline([( 'Scaler' , StandardScaler()),( 'KNN' ,
    KNeighborsClassifier())])))
pipelines.append(( 'ScaledNB' , Pipeline([( 'Scaler' , StandardScaler()),( 'NB' ,
    GaussianNB())])))
pipelines.append(( 'ScaledSVM' , Pipeline([( 'Scaler' , StandardScaler()),( 'SVM' , SVC())])))
pipelines.append(('ScaledCart',Pipeline([('Scaler',StandardScaler()),('DecisionTree',DecisionTreeClassifier(criterion='entropy'))])))
results,names = piplinecompare(models, X_train, y_train)
bxplots(results,names)

results1,names1 = piplinecompare(pipelines, X_train, y_train)
bxplots(results1,names1)

可以发现，树模型对与数据的是否标准化无影响
LDA,NB 算法 有轻微影响
LR, KNN,SVM 在进行建模之前，必须要进行合理的数据标准化，因为这对于模型训练有很大的影响.